In [1]:
import numpy as np
import pandas as pd
import math
pd.set_option('display.max_columns', 100)
pd.options.display.float_format = '{:.2f}'.format

# Outline

###  Part A. Affordable  Rentals
 1. Bring in 2015 PUMS data
 2. Assign county_id to each puma
 3. Bring in and clean, reformat county AMIs data
 4. Add variables for cost limits for ELI, VLI, LI, MI, and HI by bedroom count for each county
 5. Add count dummies for each unit based on income and unit type
 

###  Part B. Affordable Owning
 1. Calculate total monthly housing cost by adding monthly first mortgage, second/junior mortgage, and condo fees
 2. Add count dummies for each unit based on income and unit type

### Part C. Weighting Sample to get estimates
 1. Multiply each dummy by the the household weight variable
 2. Calculte standard errors for each estimate
 
### Part D. Aggregating at PUMA Level
 1. Add dummy variables for vacant "for-rent," vacant "for-sale," and recently moved in and create second dataset
 2. Aggregate at PUMA level for each dataset and export

## Part A1. Bring in 2015 PUMS data

In [2]:
variable_types = {"BDSP":"float","RMSP":"str","MV":"float","WGTP":"float","TAXP":"float"}
columns = ["PUMA","RT","BDSP","BLD","RNTP","MRGP","SMP","CONP","TEN","TAXP","VACS","VALP","GRPIP","GRNTP","HINCP","MV","WGTP"]
pums_df=pd.read_csv("2015_pums.csv", delimiter=",",usecols=columns,dtype=variable_types)
pums_df

,RT,PUMA,WGTP,BDSP,BLD,CONP,MRGP,RNTP,SMP,TEN,VACS,VALP,GRNTP,GRPIP,HINCP,MV,TAXP
0,H,5906,100.00,1.00,8.00,nan,nan,1100.00,nan,3.00,nan,nan,1120.00,26.00,52000.00,2.00,nan
1,H,8105,66.00,3.00,2.00,0.00,6200.00,nan,nan,1.00,nan,1800000.00,nan,nan,232000.00,1.00,68.00
2,H,3727,65.00,3.00,3.00,0.00,2700.00,nan,nan,1.00,nan,1200000.00,nan,nan,108800.00,4.00,68.00
3,H,6101,98.00,3.00,2.00,0.00,700.00,nan,nan,1.00,nan,375000.00,nan,nan,65000.00,4.00,57.00
4,H,3738,203.00,2.00,1.00,0.00,nan,nan,nan,2.00,nan,17000.00,nan,nan,29000.00,6.00,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155213,H,6505,65.00,3.00,2.00,240.00,nan,nan,nan,2.00,nan,330000.00,nan,nan,62600.00,6.00,68.00
155214,H,8303,67.00,4.00,2.00,0.00,2300.00,nan,nan,1.00,nan,950000.00,nan,nan,35500.00,3.00,65.00
155215,H,3717,61.00,2.00,7.00,nan,nan,1400.00,nan,3.00,nan,nan,1450.00,101.00,4500.00,1.00,nan
155216,H,2904,61.00,2.00,1.00,0.00,nan,nan,nan,2.00,nan,17000.00,nan,nan,42700.00,5.00,6.00


In [3]:
pums_df.rename(columns={"RT":"record_type","BDSP":"number_bedrooms",
    "BLD":"units_in_structure",
    "RNTP":"monthly_rent",
    "MRGP":"first_mortgage",
    "SMP":"second_mortgage",
    "CONP":"condo_fee",
    "TEN":"tenure",
    "VACS":"vacancy_status",
    "VALP":"property_value",
    "GRPIP":"gross_rent_pct_of_income",
    "GRNTP":"gross_rent",
    "HINCP":"hh_income",
    "TAXP":"prop_tax",
    "WGTP":"weight",
    "MV":"moved_in"}, inplace=True)
pums_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax
0,H,5906,100.00,1.00,8.00,nan,nan,1100.00,nan,3.00,nan,nan,1120.00,26.00,52000.00,2.00,nan
1,H,8105,66.00,3.00,2.00,0.00,6200.00,nan,nan,1.00,nan,1800000.00,nan,nan,232000.00,1.00,68.00
2,H,3727,65.00,3.00,3.00,0.00,2700.00,nan,nan,1.00,nan,1200000.00,nan,nan,108800.00,4.00,68.00
3,H,6101,98.00,3.00,2.00,0.00,700.00,nan,nan,1.00,nan,375000.00,nan,nan,65000.00,4.00,57.00
4,H,3738,203.00,2.00,1.00,0.00,nan,nan,nan,2.00,nan,17000.00,nan,nan,29000.00,6.00,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155213,H,6505,65.00,3.00,2.00,240.00,nan,nan,nan,2.00,nan,330000.00,nan,nan,62600.00,6.00,68.00
155214,H,8303,67.00,4.00,2.00,0.00,2300.00,nan,nan,1.00,nan,950000.00,nan,nan,35500.00,3.00,65.00
155215,H,3717,61.00,2.00,7.00,nan,nan,1400.00,nan,3.00,nan,nan,1450.00,101.00,4500.00,1.00,nan
155216,H,2904,61.00,2.00,1.00,0.00,nan,nan,nan,2.00,nan,17000.00,nan,nan,42700.00,5.00,6.00


## Part A2. Assign county id to each puma

In [4]:
# load in crosswalk file
crosswalk_df=pd.read_csv("C:/Users/Terner GSR/Box/Cost of Doing Nothing/Climate Impacts/Data/PUMS/PUMA_County_Crosswalk_v2.csv", delimiter=",")
crosswalk_df

,PUMA,county1,county2,county3,county4,county5,county6,county7
0,101,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
1,102,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
2,103,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
3,104,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
4,105,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
260,11103,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
261,11104,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
262,11105,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
263,11106,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# add county name column to puma file
puma_county_df=pums_df.merge(crosswalk_df, how='left', left_on = "PUMA",right_on = "PUMA")
puma_county_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7
0,H,5906,100.00,1.00,8.00,nan,nan,1100.00,nan,3.00,nan,nan,1120.00,26.00,52000.00,2.00,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN
1,H,8105,66.00,3.00,2.00,0.00,6200.00,nan,nan,1.00,nan,1800000.00,nan,nan,232000.00,1.00,68.00,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN
2,H,3727,65.00,3.00,3.00,0.00,2700.00,nan,nan,1.00,nan,1200000.00,nan,nan,108800.00,4.00,68.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN
3,H,6101,98.00,3.00,2.00,0.00,700.00,nan,nan,1.00,nan,375000.00,nan,nan,65000.00,4.00,57.00,Placer CA,NaN,NaN,NaN,NaN,NaN,NaN
4,H,3738,203.00,2.00,1.00,0.00,nan,nan,nan,2.00,nan,17000.00,nan,nan,29000.00,6.00,2.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155213,H,6505,65.00,3.00,2.00,240.00,nan,nan,nan,2.00,nan,330000.00,nan,nan,62600.00,6.00,68.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN
155214,H,8303,67.00,4.00,2.00,0.00,2300.00,nan,nan,1.00,nan,950000.00,nan,nan,35500.00,3.00,65.00,Santa Barbara CA,NaN,NaN,NaN,NaN,NaN,NaN
155215,H,3717,61.00,2.00,7.00,nan,nan,1400.00,nan,3.00,nan,nan,1450.00,101.00,4500.00,1.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN
155216,H,2904,61.00,2.00,1.00,0.00,nan,nan,nan,2.00,nan,17000.00,nan,nan,42700.00,5.00,6.00,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# remove " CA" from end of each county name
puma_county_df['County'] = puma_county_df['county1'].str.replace(r' CA', '')
puma_county_df['county2'] = puma_county_df['county2'].str.replace(r' CA', '')
puma_county_df['county3'] = puma_county_df['county3'].str.replace(r' CA', '')
puma_county_df['county4'] = puma_county_df['county4'].str.replace(r' CA', '')
puma_county_df['county5'] = puma_county_df['county5'].str.replace(r' CA', '')
puma_county_df['county6'] = puma_county_df['county6'].str.replace(r' CA', '')
puma_county_df['county7'] = puma_county_df['county7'].str.replace(r' CA', '')
puma_county_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County
0,H,5906,100.00,1.00,8.00,nan,nan,1100.00,nan,3.00,nan,nan,1120.00,26.00,52000.00,2.00,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange
1,H,8105,66.00,3.00,2.00,0.00,6200.00,nan,nan,1.00,nan,1800000.00,nan,nan,232000.00,1.00,68.00,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo
2,H,3727,65.00,3.00,3.00,0.00,2700.00,nan,nan,1.00,nan,1200000.00,nan,nan,108800.00,4.00,68.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles
3,H,6101,98.00,3.00,2.00,0.00,700.00,nan,nan,1.00,nan,375000.00,nan,nan,65000.00,4.00,57.00,Placer CA,NaN,NaN,NaN,NaN,NaN,NaN,Placer
4,H,3738,203.00,2.00,1.00,0.00,nan,nan,nan,2.00,nan,17000.00,nan,nan,29000.00,6.00,2.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155213,H,6505,65.00,3.00,2.00,240.00,nan,nan,nan,2.00,nan,330000.00,nan,nan,62600.00,6.00,68.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside
155214,H,8303,67.00,4.00,2.00,0.00,2300.00,nan,nan,1.00,nan,950000.00,nan,nan,35500.00,3.00,65.00,Santa Barbara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Barbara
155215,H,3717,61.00,2.00,7.00,nan,nan,1400.00,nan,3.00,nan,nan,1450.00,101.00,4500.00,1.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles
155216,H,2904,61.00,2.00,1.00,0.00,nan,nan,nan,2.00,nan,17000.00,nan,nan,42700.00,5.00,6.00,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern


## Part A3. Bring in and clean, reformat county AMIs data

In [7]:
#bring in 2018 county AMIs file
columns=["County","Income_Category","1","2","4","6","8"]
df = pd.read_csv("2015_amis.csv", delimiter=",", usecols=columns)
df

,County,Income_Category,1,2,4,6,8
0,Alameda County\n4-Per,Extremely Low,19650,22450,28050,32570,40890
1,Alameda County\n4-Per,Very Low Income,32750,37400,46750,54250,61750
2,Alameda County\n4-Per,Low Income,50150,57300,71600,83100,94550
3,Alameda County\n4-Per,Median Income,65450,74800,93500,108450,123400
4,Alameda County\n4-Per,Moderate Income,78550,89750,112200,130150,148100
...,...,...,...,...,...,...,...
285,Yuba County\n4-Person,Extremely Low,12500,15930,24250,32570,37600
286,Yuba County\n4-Person,Very Low Income,20800,23800,29700,34500,39250
287,Yuba County\n4-Person,Low Income,33250,38000,47500,55100,62700
288,Yuba County\n4-Person,Median Income,41600,47500,59400,68900,78400


In [8]:
#clean county names variable
df['County'] = df['County'].str.replace(r' County\n4-Person', '')
df['County'] = df['County'].str.replace(r' County\n4-Perso', '')
df['County'] = df['County'].str.replace(r' County\n4-Pers', '')
df['County'] = df['County'].str.replace(r' County\n4-Per', '')
df['County'] = df['County'].str.replace(r' County\n4-Pe', '')
df['County'] = df['County'].str.replace(r' County\n4-P', '')
df['County'] = df['County'].str.replace(r' County\n4-', '')
df['County'] = df['County'].str.replace(r' County\n4', '')
df['County'] = df['County'].str.replace(r' County\n', '')
df['County'] = df['County'].str.replace(r' County\ ' , '')
df['County'] = df['County'].str.replace(r' County', '')
df['County'] = df['County'].str.replace(r' Count', '')
df['County'] = df['County'].str.replace(r' Coun', '')
df

,County,Income_Category,1,2,4,6,8
0,Alameda,Extremely Low,19650,22450,28050,32570,40890
1,Alameda,Very Low Income,32750,37400,46750,54250,61750
2,Alameda,Low Income,50150,57300,71600,83100,94550
3,Alameda,Median Income,65450,74800,93500,108450,123400
4,Alameda,Moderate Income,78550,89750,112200,130150,148100
...,...,...,...,...,...,...,...
285,Yuba,Extremely Low,12500,15930,24250,32570,37600
286,Yuba,Very Low Income,20800,23800,29700,34500,39250
287,Yuba,Low Income,33250,38000,47500,55100,62700
288,Yuba,Median Income,41600,47500,59400,68900,78400


In [9]:
df[df["County"]=="Santa Clara"]

,County,Income_Category,1,2,4,6,8
210,Santa Clara,Extremely Low,22350,25550,31900,37050,42150
211,Santa Clara,Very Low Income,37250,42550,53150,61700,70200
212,Santa Clara,Low Income,59400,67900,84900,98450,112050
213,Santa Clara,Median Income,74400,85050,106300,123300,140300
214,Santa Clara,Moderate Income,89300,102050,127550,147950,168350


In [10]:
#create df for each income level and calculate max rents, rename variables
eli_df = df[df.Income_Category =="Extremely Low"].copy()
eli_df["1"]=eli_df["1"]*0.025
eli_df["2"]=eli_df["2"]*0.025
eli_df["4"]=eli_df["4"]*0.025
eli_df["6"]=eli_df["6"]*0.025
eli_df["8"]=eli_df["8"]*0.025
eli_df.rename(columns={"1":"ELI_studio","2":"ELI_1_br","4":"ELI_2_br","6":"ELI_3_br","8":"ELI_4_br"}, inplace=True)
eli_df.drop('Income_Category', axis=1, inplace=True)
eli_df

,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br
0,Alameda,491.25,561.25,701.25,814.25,1022.25
5,Alpine,453.75,518.75,647.50,814.25,1022.25
10,Amador,380.00,435.00,606.25,814.25,1022.25
15,Butte,308.75,398.25,606.25,814.25,940.00
20,Calaveras,368.75,421.25,606.25,814.25,1022.25
25,Colusa,308.75,398.25,606.25,814.25,968.75
30,Contra Costa,491.25,561.25,701.25,814.25,1022.25
35,Del Norte,303.75,398.25,606.25,814.25,940.00
40,El Dorado,400.00,457.50,606.25,814.25,1022.25
45,Fresno,303.75,398.25,606.25,814.25,940.00


In [11]:
vli_df = df[df.Income_Category =="Very Low Income"].copy()
vli_df["1"]=vli_df["1"]*0.025
vli_df["2"]=vli_df["2"]*0.025
vli_df["4"]=vli_df["4"]*0.025
vli_df["6"]=vli_df["6"]*0.025
vli_df["8"]=vli_df["8"]*0.025
vli_df.rename(columns={"1":"VLI_studio","2":"VLI_1_br","4":"VLI_2_br","6":"VLI_3_br","8":"VLI_4_br"}, inplace=True)
vli_df.drop('Income_Category', axis=1, inplace=True)
vli_df

,County,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br
1,Alameda,818.75,935.00,1168.75,1356.25,1543.75
6,Alpine,756.25,865.00,1080.00,1253.75,1426.25
11,Amador,633.75,723.75,903.75,1048.75,1193.75
16,Butte,513.75,587.50,733.75,851.25,968.75
21,Calaveras,615.00,702.50,877.50,1018.75,1158.75
26,Colusa,513.75,587.50,733.75,851.25,968.75
31,Contra Costa,818.75,935.00,1168.75,1356.25,1543.75
36,Del Norte,507.50,580.00,723.75,840.00,956.25
41,El Dorado,666.25,761.25,951.25,1103.75,1256.25
46,Fresno,507.50,580.00,723.75,840.00,956.25


In [12]:
li_df = df[df.Income_Category =="Low Income"].copy()
li_df["1"]=li_df["1"]*0.025
li_df["2"]=li_df["2"]*0.025
li_df["4"]=li_df["4"]*0.025
li_df["6"]=li_df["6"]*0.025
li_df["8"]=li_df["8"]*0.025
li_df.rename(columns={"1":"LI_studio","2":"LI_1_br","4":"LI_2_br","6":"LI_3_br","8":"LI_4_br"}, inplace=True)
li_df.drop('Income_Category', axis=1, inplace=True)
li_df

,County,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br
2,Alameda,1253.75,1432.50,1790.00,2077.50,2363.75
7,Alpine,1152.50,1316.25,1645.00,1908.75,2172.50
12,Amador,1012.50,1157.50,1446.25,1678.75,1910.00
17,Butte,822.50,940.00,1173.75,1362.50,1550.00
22,Calaveras,983.75,1123.75,1403.75,1628.75,1853.75
27,Colusa,822.50,940.00,1173.75,1362.50,1550.00
32,Contra Costa,1253.75,1432.50,1790.00,2077.50,2363.75
37,Del Norte,811.25,926.25,1157.50,1343.75,1528.75
42,El Dorado,1066.25,1218.75,1522.50,1766.25,2010.00
47,Fresno,811.25,926.25,1157.50,1343.75,1528.75


In [13]:
mi_df = df[df.Income_Category =="Median Income"].copy()
mi_df["1"]=mi_df["1"]*0.025
mi_df["2"]=mi_df["2"]*0.025
mi_df["4"]=mi_df["4"]*0.025
mi_df["6"]=mi_df["6"]*0.025
mi_df["8"]=mi_df["8"]*0.025
mi_df.rename(columns={"1":"MI_studio","2":"MI_1_br","4":"MI_2_br","6":"MI_3_br","8":"MI_4_br"}, inplace=True)
mi_df.drop('Income_Category', axis=1, inplace=True)
mi_df

,County,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br
3,Alameda,1636.25,1870.00,2337.50,2711.25,3085.00
8,Alpine,1661.25,1897.50,2372.50,2752.50,3131.25
13,Amador,1265.00,1446.25,1807.50,2096.25,2386.25
18,Butte,1027.50,1173.75,1467.50,1702.50,1937.50
23,Calaveras,1228.75,1403.75,1755.00,2036.25,2316.25
28,Colusa,1027.50,1173.75,1467.50,1702.50,1937.50
33,Contra Costa,1636.25,1870.00,2337.50,2711.25,3085.00
38,Del Norte,1013.75,1157.50,1447.50,1678.75,1911.25
43,El Dorado,1331.25,1522.50,1902.50,2207.50,2511.25
48,Fresno,1013.75,1157.50,1447.50,1678.75,1911.25


In [14]:
moi_df = df[df.Income_Category =="Moderate Income"].copy()
moi_df["1"]=moi_df["1"]*0.025
moi_df["2"]=moi_df["2"]*0.025
moi_df["4"]=moi_df["4"]*0.025
moi_df["6"]=moi_df["6"]*0.025
moi_df["8"]=moi_df["8"]*0.025
moi_df.rename(columns={"1":"MoI_studio","2":"MoI_1_br","4":"MoI_2_br","6":"MoI_3_br","8":"MoI_4_br"}, inplace=True)
moi_df.drop('Income_Category', axis=1, inplace=True)
moi_df

,County,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
4,Alameda,1963.75,2243.75,2805.00,3253.75,3702.50
9,Alpine,1993.75,2277.50,2847.50,3302.50,3758.75
14,Amador,1517.50,1735.00,2168.75,2516.25,2862.50
19,Butte,1232.50,1408.75,1761.25,2042.50,2325.00
24,Calaveras,1473.75,1685.00,2106.25,2443.75,2780.00
29,Colusa,1232.50,1408.75,1761.25,2042.50,2325.00
34,Contra Costa,1963.75,2243.75,2805.00,3253.75,3702.50
39,Del Norte,1216.25,1390.00,1737.50,2015.00,2293.75
44,El Dorado,1597.50,1826.25,2282.50,2647.50,3012.50
49,Fresno,1216.25,1390.00,1737.50,2015.00,2293.75


In [15]:
#combine dfs
all_amis_df=eli_df.merge(vli_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(li_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(mi_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(moi_df, how='left', left_on = "County",right_on = "County")
all_amis_df

,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
0,Alameda,491.25,561.25,701.25,814.25,1022.25,818.75,935.00,1168.75,1356.25,1543.75,1253.75,1432.50,1790.00,2077.50,2363.75,1636.25,1870.00,2337.50,2711.25,3085.00,1963.75,2243.75,2805.00,3253.75,3702.50
1,Alpine,453.75,518.75,647.50,814.25,1022.25,756.25,865.00,1080.00,1253.75,1426.25,1152.50,1316.25,1645.00,1908.75,2172.50,1661.25,1897.50,2372.50,2752.50,3131.25,1993.75,2277.50,2847.50,3302.50,3758.75
2,Amador,380.00,435.00,606.25,814.25,1022.25,633.75,723.75,903.75,1048.75,1193.75,1012.50,1157.50,1446.25,1678.75,1910.00,1265.00,1446.25,1807.50,2096.25,2386.25,1517.50,1735.00,2168.75,2516.25,2862.50
3,Butte,308.75,398.25,606.25,814.25,940.00,513.75,587.50,733.75,851.25,968.75,822.50,940.00,1173.75,1362.50,1550.00,1027.50,1173.75,1467.50,1702.50,1937.50,1232.50,1408.75,1761.25,2042.50,2325.00
4,Calaveras,368.75,421.25,606.25,814.25,1022.25,615.00,702.50,877.50,1018.75,1158.75,983.75,1123.75,1403.75,1628.75,1853.75,1228.75,1403.75,1755.00,2036.25,2316.25,1473.75,1685.00,2106.25,2443.75,2780.00
5,Colusa,308.75,398.25,606.25,814.25,968.75,513.75,587.50,733.75,851.25,968.75,822.50,940.00,1173.75,1362.50,1550.00,1027.50,1173.75,1467.50,1702.50,1937.50,1232.50,1408.75,1761.25,2042.50,2325.00
6,Contra Costa,491.25,561.25,701.25,814.25,1022.25,818.75,935.00,1168.75,1356.25,1543.75,1253.75,1432.50,1790.00,2077.50,2363.75,1636.25,1870.00,2337.50,2711.25,3085.00,1963.75,2243.75,2805.00,3253.75,3702.50
7,Del Norte,303.75,398.25,606.25,814.25,940.00,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75
8,El Dorado,400.00,457.50,606.25,814.25,1022.25,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50
9,Fresno,303.75,398.25,606.25,814.25,940.00,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75


## Part A4. Add count dummies for each unit based on income and unit type

In [16]:
# merge datasets
units_df=puma_county_df.merge(all_amis_df, how='left', left_on = "County",right_on = "County")
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
0,H,5906,100.00,1.00,8.00,nan,nan,1100.00,nan,3.00,nan,nan,1120.00,26.00,52000.00,2.00,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,506.25,578.75,722.50,838.75,1022.25,843.75,963.75,1203.75,1397.50,1590.00,1348.75,1541.25,1926.25,2235.00,2543.75,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75
1,H,8105,66.00,3.00,2.00,0.00,6200.00,nan,nan,1.00,nan,1800000.00,nan,nan,232000.00,1.00,68.00,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo,616.25,703.75,878.75,1020.00,1160.00,1026.25,1172.50,1465.00,1700.00,1935.00,1642.50,1877.50,2346.25,2722.50,3097.50,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75
2,H,3727,65.00,3.00,3.00,0.00,2700.00,nan,nan,1.00,nan,1200000.00,nan,nan,108800.00,4.00,68.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,814.25,1022.25,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25
3,H,6101,98.00,3.00,2.00,0.00,700.00,nan,nan,1.00,nan,375000.00,nan,nan,65000.00,4.00,57.00,Placer CA,NaN,NaN,NaN,NaN,NaN,NaN,Placer,400.00,457.50,606.25,814.25,1022.25,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50
4,H,3738,203.00,2.00,1.00,0.00,nan,nan,nan,2.00,nan,17000.00,nan,nan,29000.00,6.00,2.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,814.25,1022.25,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155213,H,6505,65.00,3.00,2.00,240.00,nan,nan,nan,2.00,nan,330000.00,nan,nan,62600.00,6.00,68.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,606.25,814.25,1022.25,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75
155214,H,8303,67.00,4.00,2.00,0.00,2300.00,nan,nan,1.00,nan,950000.00,nan,nan,35500.00,3.00,65.00,Santa Barbara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Barbara,421.25,481.25,606.25,814.25,1022.25,702.50,802.50,1002.50,1163.75,1323.75,1123.75,1283.75,1603.75,1861.25,2117.50,1320.00,1507.50,1885.00,2186.25,2488.75,1583.75,1810.00,2262.50,2625.00,2986.25
155215,H,3717,61.00,2.00,7.00,nan,nan,1400.00,nan,3.00,nan,nan,1450.00,101.00,4500.00,1.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,814.25,1022.25,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25
155216,H,2904,61.00,2.00,1.00,0.00,nan,nan,nan,2.00,nan,17000.00,nan,nan,42700.00,5.00,6.00,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern,303.75,398.25,606.25,814.25,940.00,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75


In [17]:
# create filter masks for each affordability and unit type
ELI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["ELI_studio"])
ELI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["ELI_1_br"])
ELI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["ELI_2_br"])
ELI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["ELI_3_br"])
ELI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["ELI_4_br"])
VLI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["VLI_studio"])
VLI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["VLI_1_br"])
VLI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["VLI_2_br"])
VLI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["VLI_3_br"])
VLI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["VLI_4_br"])
LI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["LI_studio"])
LI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["LI_1_br"])
LI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["LI_2_br"])
LI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["LI_3_br"])
LI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["LI_4_br"])
MI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["MI_studio"])
MI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["MI_1_br"])
MI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["MI_2_br"])
MI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["MI_3_br"])
MI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["MI_4_br"])
MoI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["MoI_studio"])
MoI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["MoI_1_br"])
MoI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["MoI_2_br"])
MoI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["MoI_3_br"])
MoI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["MoI_4_br"])
HI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']>=units_df["MoI_studio"])
HI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']>=units_df["MoI_1_br"])
HI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']>=units_df["MoI_2_br"])
HI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']>=units_df["MoI_3_br"])
HI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']>=units_df["MoI_4_br"])

# apply filter masks to create dummy variables
units_df['rent_ELI_studio_count']=np.where(ELI_studio_mask,1,0)
units_df['rent_ELI_1_br_count']=np.where(ELI_1_br_mask,1,0)
units_df['rent_ELI_2_br_count']=np.where(ELI_2_br_mask,1,0)
units_df['rent_ELI_3_br_count']=np.where(ELI_3_br_mask,1,0)
units_df['rent_ELI_4_br_count']=np.where(ELI_4_br_mask,1,0)
units_df['rent_VLI_studio_count']=np.where(VLI_studio_mask,1,0)
units_df['rent_VLI_1_br_count']=np.where(VLI_1_br_mask,1,0)
units_df['rent_VLI_2_br_count']=np.where(VLI_2_br_mask,1,0)
units_df['rent_VLI_3_br_count']=np.where(VLI_3_br_mask,1,0)
units_df['rent_VLI_4_br_count']=np.where(VLI_4_br_mask,1,0)
units_df['rent_LI_studio_count']=np.where(LI_studio_mask,1,0)
units_df['rent_LI_1_br_count']=np.where(LI_1_br_mask,1,0)
units_df['rent_LI_2_br_count']=np.where(LI_2_br_mask,1,0)
units_df['rent_LI_3_br_count']=np.where(LI_3_br_mask,1,0)
units_df['rent_LI_4_br_count']=np.where(LI_4_br_mask,1,0)
units_df['rent_MI_studio_count']=np.where(MI_studio_mask,1,0)
units_df['rent_MI_1_br_count']=np.where(MI_1_br_mask,1,0)
units_df['rent_MI_2_br_count']=np.where(MI_2_br_mask,1,0)
units_df['rent_MI_3_br_count']=np.where(MI_3_br_mask,1,0)
units_df['rent_MI_4_br_count']=np.where(MI_4_br_mask,1,0)
units_df['rent_MoI_studio_count']=np.where(MoI_studio_mask,1,0)
units_df['rent_MoI_1_br_count']=np.where(MoI_1_br_mask,1,0)
units_df['rent_MoI_2_br_count']=np.where(MoI_2_br_mask,1,0)
units_df['rent_MoI_3_br_count']=np.where(MoI_3_br_mask,1,0)
units_df['rent_MoI_4_br_count']=np.where(MoI_4_br_mask,1,0)
units_df['rent_HI_studio_count']=np.where(HI_studio_mask,1,0)
units_df['rent_HI_1_br_count']=np.where(HI_1_br_mask,1,0)
units_df['rent_HI_2_br_count']=np.where(HI_2_br_mask,1,0)
units_df['rent_HI_3_br_count']=np.where(HI_3_br_mask,1,0)
units_df['rent_HI_4_br_count']=np.where(HI_4_br_mask,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count
0,H,5906,100.00,1.00,8.00,nan,nan,1100.00,nan,3.00,nan,nan,1120.00,26.00,52000.00,2.00,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,506.25,578.75,722.50,838.75,1022.25,843.75,963.75,1203.75,1397.50,1590.00,1348.75,1541.25,1926.25,2235.00,2543.75,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,H,8105,66.00,3.00,2.00,0.00,6200.00,nan,nan,1.00,nan,1800000.00,nan,nan,232000.00,1.00,68.00,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo,616.25,703.75,878.75,1020.00,1160.00,1026.25,1172.50,1465.00,1700.00,1935.00,1642.50,1877.50,2346.25,2722.50,3097.50,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,H,3727,65.00,3.00,3.00,0.00,2700.00,nan,nan,1.00,nan,1200000.00,nan,nan,108800.00,4.00,68.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,814.25,1022.25,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,H,6101,98.00,3.00,2.00,0.00,700.00,nan,nan,1.00,nan,375000.00,nan,nan,65000.00,4.00,57.00,Placer CA,NaN,NaN,NaN,NaN,NaN,NaN,Placer,400.00,457.50,606.25,814.25,1022.25,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H,3738,203.00,2.00,1.00,0.00,nan,nan,nan,2.00,nan,17000.00,nan,nan,29000.00,6.00,2.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,814.25,1022.25,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155213,H,6505,65.00,3.00,2.00,240.00,nan,nan,nan,2.00,nan,330000.00,nan,nan,62600.00,6.00,68.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,606.25,814.25,1022.25,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
155214,H,8303,67.00,4.00,2.00,0.00,2300.00,nan,nan,1.00,nan,950000.00,nan,nan,35500.00,3.00,65.00,Santa Barbara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Barbara,421.25,481.25,606.25,814.25,1022.25,702.50,802.50,1002.50,1163.75

## Part B1. Calculate total monthly housing cost by adding monthly first mortgage, second/junior mortgage, and condo fees

In [18]:
units_df['own_monthly_cost']=units_df['first_mortgage'] + units_df['second_mortgage'] + units_df['condo_fee'] + (units_df['prop_tax']/12)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost
0,H,5906,100.00,1.00,8.00,nan,nan,1100.00,nan,3.00,nan,nan,1120.00,26.00,52000.00,2.00,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,506.25,578.75,722.50,838.75,1022.25,843.75,963.75,1203.75,1397.50,1590.00,1348.75,1541.25,1926.25,2235.00,2543.75,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,nan
1,H,8105,66.00,3.00,2.00,0.00,6200.00,nan,nan,1.00,nan,1800000.00,nan,nan,232000.00,1.00,68.00,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo,616.25,703.75,878.75,1020.00,1160.00,1026.25,1172.50,1465.00,1700.00,1935.00,1642.50,1877.50,2346.25,2722.50,3097.50,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
2,H,3727,65.00,3.00,3.00,0.00,2700.00,nan,nan,1.00,nan,1200000.00,nan,nan,108800.00,4.00,68.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,814.25,1022.25,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
3,H,6101,98.00,3.00,2.00,0.00,700.00,nan,nan,1.00,nan,375000.00,nan,nan,65000.00,4.00,57.00,Placer CA,NaN,NaN,NaN,NaN,NaN,NaN,Placer,400.00,457.50,606.25,814.25,1022.25,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
4,H,3738,203.00,2.00,1.00,0.00,nan,nan,nan,2.00,nan,17000.00,nan,nan,29000.00,6.00,2.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,814.25,1022.25,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155213,H,6505,65.00,3.00,2.00,240.00,nan,nan,nan,2.00,nan,330000.00,nan,nan,62600.00,6.00,68.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,606.25,814.25,1022.25,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
155214,H,8303,67.00,4.00,2.00,0.00,2300.00,nan,nan,1.00,nan,950000.00,nan,nan,35500.00,3.00,65.00,Santa Barbara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Barbara,421.25,481.25,606.25

# Part B2. Add count dummies for each unit based on income and unit type

In [19]:
# create filter masks for each affordability and unit type
ELI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["ELI_studio"])
ELI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["ELI_1_br"])
ELI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["ELI_2_br"])
ELI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["ELI_3_br"])
ELI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["ELI_4_br"])
VLI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["VLI_studio"])
VLI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["VLI_1_br"])
VLI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["VLI_2_br"])
VLI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["VLI_3_br"])
VLI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["VLI_4_br"])
LI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["LI_studio"])
LI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["LI_1_br"])
LI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["LI_2_br"])
LI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["LI_3_br"])
LI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["LI_4_br"])
MI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["MI_studio"])
MI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["MI_1_br"])
MI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["MI_2_br"])
MI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["MI_3_br"])
MI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["MI_4_br"])
MoI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["MoI_studio"])
MoI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["MoI_1_br"])
MoI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["MoI_2_br"])
MoI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["MoI_3_br"])
MoI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["MoI_4_br"])
HI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']>=units_df["MoI_studio"])
HI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']>=units_df["MoI_1_br"])
HI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']>=units_df["MoI_2_br"])
HI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']>=units_df["MoI_3_br"])
HI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']>=units_df["MoI_4_br"])

# apply filter masks to create dummy variables
units_df['own_ELI_studio_count']=np.where(ELI_studio_mask,1,0)
units_df['own_ELI_1_br_count']=np.where(ELI_1_br_mask,1,0)
units_df['own_ELI_2_br_count']=np.where(ELI_2_br_mask,1,0)
units_df['own_ELI_3_br_count']=np.where(ELI_3_br_mask,1,0)
units_df['own_ELI_4_br_count']=np.where(ELI_4_br_mask,1,0)
units_df['own_VLI_studio_count']=np.where(VLI_studio_mask,1,0)
units_df['own_VLI_1_br_count']=np.where(VLI_1_br_mask,1,0)
units_df['own_VLI_2_br_count']=np.where(VLI_2_br_mask,1,0)
units_df['own_VLI_3_br_count']=np.where(VLI_3_br_mask,1,0)
units_df['own_VLI_4_br_count']=np.where(VLI_4_br_mask,1,0)
units_df['own_LI_studio_count']=np.where(LI_studio_mask,1,0)
units_df['own_LI_1_br_count']=np.where(LI_1_br_mask,1,0)
units_df['own_LI_2_br_count']=np.where(LI_2_br_mask,1,0)
units_df['own_LI_3_br_count']=np.where(LI_3_br_mask,1,0)
units_df['own_LI_4_br_count']=np.where(LI_4_br_mask,1,0)
units_df['own_MI_studio_count']=np.where(MI_studio_mask,1,0)
units_df['own_MI_1_br_count']=np.where(MI_1_br_mask,1,0)
units_df['own_MI_2_br_count']=np.where(MI_2_br_mask,1,0)
units_df['own_MI_3_br_count']=np.where(MI_3_br_mask,1,0)
units_df['own_MI_4_br_count']=np.where(MI_4_br_mask,1,0)
units_df['own_MoI_studio_count']=np.where(MoI_studio_mask,1,0)
units_df['own_MoI_1_br_count']=np.where(MoI_1_br_mask,1,0)
units_df['own_MoI_2_br_count']=np.where(MoI_2_br_mask,1,0)
units_df['own_MoI_3_br_count']=np.where(MoI_3_br_mask,1,0)
units_df['own_MoI_4_br_count']=np.where(MoI_4_br_mask,1,0)
units_df['own_HI_studio_count']=np.where(HI_studio_mask,1,0)
units_df['own_HI_1_br_count']=np.where(HI_1_br_mask,1,0)
units_df['own_HI_2_br_count']=np.where(HI_2_br_mask,1,0)
units_df['own_HI_3_br_count']=np.where(HI_3_br_mask,1,0)
units_df['own_HI_4_br_count']=np.where(HI_4_br_mask,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
0,H,5906,100.00,1.00,8.00,nan,nan,1100.00,nan,3.00,nan,nan,1120.00,26.00,52000.00,2.00,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,506.25,578.75,722.50,838.75,1022.25,843.75,963.75,1203.75,1397.50,1590.00,1348.75,1541.25,1926.25,2235.00,2543.75,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75,...,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H,8105,66.00,3.00,2.00,0.00,6200.00,nan,nan,1.00,nan,1800000.00,nan,nan,232000.00,1.00,68.00,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo,616.25,703.75,878.75,1020.00,1160.00,1026.25,1172.50,1465.00,1700.00,1935.00,1642.50,1877.50,2346.25,2722.50,3097.50,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,H,3727,65.00,3.00,3.00,0.00,2700.00,nan,nan,1.00,nan,1200000.00,nan,nan,108800.00,4.00,68.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,814.25,1022.25,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,H,6101,98.00,3.00,2.00,0.00,700.00,nan,nan,1.00,nan,375000.00,nan,nan,65000.00,4.00,57.00,Placer CA,NaN,NaN,NaN,NaN,NaN,NaN,Placer,400.00,457.50,606.25,814.25,1022.25,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H,3738,203.00,2.00,1.00,0.00,nan,nan,nan,2.00,nan,17000.00,nan,nan,29000.00,6.00,2.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,814.25,1022.25,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

# Part C1. Multiply each dummy by the the household weight variable

In [20]:
units_df.rent_ELI_studio_count=units_df.rent_ELI_studio_count*units_df.weight
units_df.rent_ELI_1_br_count=units_df.rent_ELI_1_br_count*units_df.weight
units_df.rent_ELI_2_br_count=units_df.rent_ELI_2_br_count*units_df.weight
units_df.rent_ELI_3_br_count=units_df.rent_ELI_3_br_count*units_df.weight
units_df.rent_ELI_4_br_count=units_df.rent_ELI_4_br_count*units_df.weight
units_df.rent_VLI_studio_count=units_df.rent_VLI_studio_count*units_df.weight
units_df.rent_VLI_1_br_count=units_df.rent_VLI_1_br_count*units_df.weight
units_df.rent_VLI_2_br_count=units_df.rent_VLI_2_br_count*units_df.weight
units_df.rent_VLI_3_br_count=units_df.rent_VLI_3_br_count*units_df.weight
units_df.rent_VLI_4_br_count=units_df.rent_VLI_4_br_count*units_df.weight
units_df.rent_LI_studio_count=units_df.rent_LI_studio_count*units_df.weight
units_df.rent_LI_1_br_count=units_df.rent_LI_1_br_count*units_df.weight
units_df.rent_LI_2_br_count=units_df.rent_LI_2_br_count*units_df.weight
units_df.rent_LI_3_br_count=units_df.rent_LI_3_br_count*units_df.weight
units_df.rent_LI_4_br_count=units_df.rent_LI_4_br_count*units_df.weight
units_df.rent_MI_studio_count=units_df.rent_MI_studio_count*units_df.weight
units_df.rent_MI_1_br_count=units_df.rent_MI_1_br_count*units_df.weight
units_df.rent_MI_2_br_count=units_df.rent_MI_2_br_count*units_df.weight
units_df.rent_MI_3_br_count=units_df.rent_MI_3_br_count*units_df.weight
units_df.rent_MI_4_br_count=units_df.rent_MI_4_br_count*units_df.weight
units_df.rent_MoI_studio_count=units_df.rent_MoI_studio_count*units_df.weight
units_df.rent_MoI_1_br_count=units_df.rent_MoI_1_br_count*units_df.weight
units_df.rent_MoI_2_br_count=units_df.rent_MoI_2_br_count*units_df.weight
units_df.rent_MoI_3_br_count=units_df.rent_MoI_3_br_count*units_df.weight
units_df.rent_MoI_4_br_count=units_df.rent_MoI_4_br_count*units_df.weight
units_df.rent_HI_studio_count=units_df.rent_HI_studio_count*units_df.weight
units_df.rent_HI_1_br_count=units_df.rent_HI_1_br_count*units_df.weight
units_df.rent_HI_2_br_count=units_df.rent_HI_2_br_count*units_df.weight
units_df.rent_HI_3_br_count=units_df.rent_HI_3_br_count*units_df.weight
units_df.rent_HI_4_br_count=units_df.rent_HI_4_br_count*units_df.weight
units_df.own_ELI_studio_count=units_df.own_ELI_studio_count*units_df.weight
units_df.own_ELI_1_br_count=units_df.own_ELI_1_br_count*units_df.weight
units_df.own_ELI_2_br_count=units_df.own_ELI_2_br_count*units_df.weight
units_df.own_ELI_3_br_count=units_df.own_ELI_3_br_count*units_df.weight
units_df.own_ELI_4_br_count=units_df.own_ELI_4_br_count*units_df.weight
units_df.own_VLI_studio_count=units_df.own_VLI_studio_count*units_df.weight
units_df.own_VLI_1_br_count=units_df.own_VLI_1_br_count*units_df.weight
units_df.own_VLI_2_br_count=units_df.own_VLI_2_br_count*units_df.weight
units_df.own_VLI_3_br_count=units_df.own_VLI_3_br_count*units_df.weight
units_df.own_VLI_4_br_count=units_df.own_VLI_4_br_count*units_df.weight
units_df.own_LI_studio_count=units_df.own_LI_studio_count*units_df.weight
units_df.own_LI_1_br_count=units_df.own_LI_1_br_count*units_df.weight
units_df.own_LI_2_br_count=units_df.own_LI_2_br_count*units_df.weight
units_df.own_LI_3_br_count=units_df.own_LI_3_br_count*units_df.weight
units_df.own_LI_4_br_count=units_df.own_LI_4_br_count*units_df.weight
units_df.own_MI_studio_count=units_df.own_MI_studio_count*units_df.weight
units_df.own_MI_1_br_count=units_df.own_MI_1_br_count*units_df.weight
units_df.own_MI_2_br_count=units_df.own_MI_2_br_count*units_df.weight
units_df.own_MI_3_br_count=units_df.own_MI_3_br_count*units_df.weight
units_df.own_MI_4_br_count=units_df.own_MI_4_br_count*units_df.weight
units_df.own_MoI_studio_count=units_df.own_MoI_studio_count*units_df.weight
units_df.own_MoI_1_br_count=units_df.own_MoI_1_br_count*units_df.weight
units_df.own_MoI_2_br_count=units_df.own_MoI_2_br_count*units_df.weight
units_df.own_MoI_3_br_count=units_df.own_MoI_3_br_count*units_df.weight
units_df.own_MoI_4_br_count=units_df.own_MoI_4_br_count*units_df.weight
units_df.own_HI_studio_count=units_df.own_HI_studio_count*units_df.weight
units_df.own_HI_1_br_count=units_df.own_HI_1_br_count*units_df.weight
units_df.own_HI_2_br_count=units_df.own_HI_2_br_count*units_df.weight
units_df.own_HI_3_br_count=units_df.own_HI_3_br_count*units_df.weight
units_df.own_HI_4_br_count=units_df.own_HI_4_br_count*units_df.weight
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
0,H,5906,100.00,1.00,8.00,nan,nan,1100.00,nan,3.00,nan,nan,1120.00,26.00,52000.00,2.00,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,506.25,578.75,722.50,838.75,1022.25,843.75,963.75,1203.75,1397.50,1590.00,1348.75,1541.25,1926.25,2235.00,2543.75,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75,...,100.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,H,8105,66.00,3.00,2.00,0.00,6200.00,nan,nan,1.00,nan,1800000.00,nan,nan,232000.00,1.00,68.00,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo,616.25,703.75,878.75,1020.00,1160.00,1026.25,1172.50,1465.00,1700.00,1935.00,1642.50,1877.50,2346.25,2722.50,3097.50,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,H,3727,65.00,3.00,3.00,0.00,2700.00,nan,nan,1.00,nan,1200000.00,nan,nan,108800.00,4.00,68.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,814.25,1022.25,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,H,6101,98.00,3.00,2.00,0.00,700.00,nan,nan,1.00,nan,375000.00,nan,nan,65000.00,4.00,57.00,Placer CA,NaN,NaN,NaN,NaN,NaN,NaN,Placer,400.00,457.50,606.25,814.25,1022.25,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,H,3738,203.00,2.00,1.00,0.00,nan,nan,nan,2.00,nan,17000.00,nan,nan,29000.00,6.00,2.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,814.25,1022.25,747.50,855.00

In [21]:
units_df[units_df.rent_HI_4_br_count>0]

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
154,H,3725,82.00,4.00,2.00,nan,nan,2700.00,nan,3.00,nan,nan,3540.00,62.00,68000.00,1.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,814.25,1022.25,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,82.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
599,H,5902,132.00,4.00,2.00,nan,nan,3700.00,nan,3.00,nan,nan,3916.00,9.00,531800.00,3.00,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,506.25,578.75,722.50,838.75,1022.25,843.75,963.75,1203.75,1397.50,1590.00,1348.75,1541.25,1926.25,2235.00,2543.75,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,132.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
900,H,5904,37.00,4.00,2.00,nan,nan,3700.00,nan,3.00,nan,nan,3908.00,10.00,483000.00,1.00,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,506.25,578.75,722.50,838.75,1022.25,843.75,963.75,1203.75,1397.50,1590.00,1348.75,1541.25,1926.25,2235.00,2543.75,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,37.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3673,H,7114,63.00,4.00,2.00,nan,nan,2700.00,nan,3.00,nan,nan,2888.00,101.00,26700.00,3.00,nan,San Bernardino CA,NaN,NaN,NaN,NaN,NaN,NaN,San Bernardino,352.50,402.50,606.25,814.25,1022.25,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,63.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3797,H,3763,94.00,4.00,2.00,nan,nan,3700.00,nan,3.00,nan,nan,4303.00,39.00,130790.00,1.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,814.25,1022.25,747.

# Part D1. Add dummy variables for vacant "for-rent," vacant "for-sale," and recently moved in

In [22]:
# create dummy variables for vacant for rent, vacant for sale, rented in the last year, and sold in the last year
units_df["vacant_for_rent"]=np.where(units_df['vacancy_status']==1,1,0)
units_df["vacant_for_sale"]=np.where(units_df['vacancy_status']==3,1,0)
units_df["moved_last_year"]=np.where(units_df['moved_in']==1,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count,vacant_for_rent,vacant_for_sale,moved_last_year
0,H,5906,100.00,1.00,8.00,nan,nan,1100.00,nan,3.00,nan,nan,1120.00,26.00,52000.00,2.00,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,506.25,578.75,722.50,838.75,1022.25,843.75,963.75,1203.75,1397.50,1590.00,1348.75,1541.25,1926.25,2235.00,2543.75,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75,...,0.00,0.00,100.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
1,H,8105,66.00,3.00,2.00,0.00,6200.00,nan,nan,1.00,nan,1800000.00,nan,nan,232000.00,1.00,68.00,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo,616.25,703.75,878.75,1020.00,1160.00,1026.25,1172.50,1465.00,1700.00,1935.00,1642.50,1877.50,2346.25,2722.50,3097.50,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
2,H,3727,65.00,3.00,3.00,0.00,2700.00,nan,nan,1.00,nan,1200000.00,nan,nan,108800.00,4.00,68.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,814.25,1022.25,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
3,H,6101,98.00,3.00,2.00,0.00,700.00,nan,nan,1.00,nan,375000.00,nan,nan,65000.00,4.00,57.00,Placer CA,NaN,NaN,NaN,NaN,NaN,NaN,Placer,400.00,457.50,606.25,814.25,1022.25,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
4,H,3738,203.00,2.00,1.00,0.00,nan,nan,nan,2.00,nan,17000.00,nan,nan,29000.00,6.00,2.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,814.25,1022.25,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.5

In [23]:
#filter for only currently listed or recently moved into units
recent_mask = (units_df.vacant_for_rent==1)|(units_df.vacant_for_sale==1)|(units_df.moved_last_year==1)
recent_units_df = units_df[recent_mask]
recent_units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count,vacant_for_rent,vacant_for_sale,moved_last_year
1,H,8105,66.00,3.00,2.00,0.00,6200.00,nan,nan,1.00,nan,1800000.00,nan,nan,232000.00,1.00,68.00,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo,616.25,703.75,878.75,1020.00,1160.00,1026.25,1172.50,1465.00,1700.00,1935.00,1642.50,1877.50,2346.25,2722.50,3097.50,1802.50,2060.00,2575.00,2987.50,3398.75,2162.50,2472.50,3090.00,3585.00,4078.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
10,H,3719,56.00,1.00,6.00,nan,nan,1700.00,nan,3.00,nan,nan,1850.00,31.00,70730.00,1.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,814.25,1022.25,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,56.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
21,H,3721,60.00,1.00,6.00,nan,nan,1200.00,nan,3.00,nan,nan,1290.00,42.00,37000.00,1.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,814.25,1022.25,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,60.00,0.00,0.00,0.00,0.00,60.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
35,H,101,45.00,3.00,2.00,0.00,3800.00,nan,nan,1.00,nan,800000.00,nan,nan,140000.00,1.00,68.00,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN,Alameda,491.25,561.25,701.25,814.25,1022.25,818.75,935.00,1168.75,1356.25,1543.75,1253.75,1432.50,1790.00,2077.50,2363.75,1636.25,1870.00,2337.50,2711.25,3085.00,1963.75,2243.75,2805.00,3253.75,3702.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
38,H,7703,50.00,1.00,5.00,nan,nan,230.00,nan,3.00,nan,nan,270.00,27.00,12000.00,1.00,nan,San Joaquin CA,NaN,NaN,NaN,NaN,NaN,NaN,San Joaquin,348.75,398.75,606.25,814.25,983.75,581.25,663.75,828.75,962.50,1095.00,928.75,1061.25

# Part D2. Aggregating at the PUMA level

In [24]:
# for all units
puma_sums_df = units_df.groupby("PUMA").sum()
puma_counts_df = puma_sums_df[["rent_ELI_studio_count","rent_ELI_1_br_count",
    "rent_ELI_2_br_count",
    "rent_ELI_3_br_count",
    "rent_ELI_4_br_count",
    "rent_VLI_studio_count",
    "rent_VLI_1_br_count",
    "rent_VLI_2_br_count",
    "rent_VLI_3_br_count",
    "rent_VLI_4_br_count",
    "rent_LI_studio_count",
    "rent_LI_1_br_count",
    "rent_LI_2_br_count",
    "rent_LI_3_br_count",
    "rent_LI_4_br_count",
    "rent_MI_studio_count",
    "rent_MI_1_br_count",
    "rent_MI_2_br_count",
    "rent_MI_3_br_count",
    "rent_MI_4_br_count",
    "rent_MoI_studio_count",
    "rent_MoI_1_br_count",
    "rent_MoI_2_br_count",
    "rent_MoI_3_br_count",
    "rent_MoI_4_br_count",
    "rent_HI_studio_count",
    "rent_HI_1_br_count",
    "rent_HI_2_br_count",
    "rent_HI_3_br_count",
    "rent_HI_4_br_count",
    "own_ELI_studio_count",
    "own_ELI_1_br_count",
    "own_ELI_2_br_count",
    "own_ELI_3_br_count",
    "own_ELI_4_br_count",
    "own_VLI_studio_count",
    "own_VLI_1_br_count",
    "own_VLI_2_br_count",
    "own_VLI_3_br_count",
    "own_VLI_4_br_count",
    "own_LI_studio_count",
    "own_LI_1_br_count",
    "own_LI_2_br_count",
    "own_LI_3_br_count",
    "own_LI_4_br_count",
    "own_MI_studio_count",
    "own_MI_1_br_count",
    "own_MI_2_br_count",
    "own_MI_3_br_count",
    "own_MI_4_br_count",
    "own_MoI_studio_count",
    "own_MoI_1_br_count",
    "own_MoI_2_br_count",
    "own_MoI_3_br_count",
    "own_MoI_4_br_count",
    "own_HI_studio_count",
    "own_HI_1_br_count",
    "own_HI_2_br_count",
    "own_HI_3_br_count",
    "own_HI_4_br_count"]].copy()
puma_counts_df

,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
PUMA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101,503.00,1196.00,370.00,186.00,0.00,1632.00,2875.00,1668.00,605.00,144.00,2945.00,7068.00,5661.00,1332.00,341.00,4247.00,9544.00,8699.00,2461.00,502.00,4609.00,11214.00,9424.00,2637.00,813.00,157.00,278.00,71.00,302.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,129.00,0.00,0.00,0.00,142.00,526.00,58.00,0.00,0.00,233.00,630.00,97.00,0.00,0.00,369.00,926.00,377.00,0.00,0.00,356.00,487.00,309.00
102,2771.00,2817.00,1720.00,1453.00,391.00,5823.00,7656.00,4993.00,3564.00,541.00,9305.00,13766.00,11442.00,6224.00,1070.00,10727.00,17036.00,13904.00,6473.00,1217.00,11161.00,18926.00,14756.00,6473.00,1438.00,528.00,1145.00,1132.00,248.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,65.00,64.00,137.00,0.00,34.00,211.00,164.00,252.00,0.00,149.00,329.00,487.00,317.00,0.00,305.00,329.00,548.00,533.00,78.00,237.00,37.00,68.00,73.00
103,70.00,852.00,630.00,236.00,153.00,729.00,2006.00,1496.00,933.00,153.00,1728.00,5170.00,3906.00,1914.00,198.00,2160.00,7221.00,4966.00,2440.00,310.00,2160.00,8031.00,5718.00,3169.00,338.00,0.00,698.00,60.00,314.00,0.00,0.00,0.00,0.00,107.00,155.00,0.00,0.00,57.00,249.00,155.00,0.00,0.00,199.00,817.00,778.00,0.00,0.00,534.00,1593.00,1164.00,0.00,0.00,699.00,1775.00,1429.00,0.00,0.00,265.00,596.00,370.00
104,195.00,2129.00,1594.00,1846.00,146.00,1162.00,4222.00,6059.00,3214.00,971.00,1696.00,6083.00,10419.00,4961.00,1320.00,1735.00,6083.00,11318.00,5129.00,1320.00,1812.00,6191.00,11318.00,5129.00,1320.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,137.00,0.00,0.00,0.00,0.00,170.00,47.00,0.00,29.00,131.00,170.00,158.00,0.00,29.00,392.00,326.00,246.00,0.00,29.00,570.00,326.00,246.00,0.00,0.00,267.00,44.00,75.00
105,175.00,185.00,499.00,375.00,220.00,175.00,1946.00,3027.00,1080.00,484.00,1787.00,8807.00,10438.00,2733.00,656.00,1995.00,10383.00,12557.00,3355.00,731.00,2105.00,10573.00,13158.00,3520.00,1173.00,223.00,68.00,454.00,131.00,0.00,0.00,0.00,0.00,0.00,113.00,0.00,0.00,0.00,508.00,181.00,0.00,0.00,63.00,583.00,409.00,0.00,0.00,298.00,1067.00,555.00,0.00,0.00,358.00,1422.00,555.00,55.00,0.00,306.00,354.00,320.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11103,0.00,1090.00,1144.00,392.00,57.00,617.00,1815.00,2641.00,1137.00,310.00,1460.00,3876.00,9065.00,5063.00,2859.00,1743.00,4324.00,9756.00,5834.00,3103.00,1743.00,4324.00,10000.00,5913.00,3367.00,0.00,0.00,55.00,0.00,0.00,0.00,0.00,0.00,0.00,44.00,0.00,0.00,109.00,187.00,44.00,0.00,0.00,311.00,785.00,455.00,0.00,0.00,311.00,946.00,455.00,0.00,0.00,311.00,1007.00,455.00,0.00,0.00,0.00,63.00,56.00
11104,109.00,659.00,478.00,335.00,265.00,236.00,1332.00,1233.00,

In [25]:
#export it
puma_counts_df.to_csv("weighted_puma_counts.csv")

In [26]:
# for recent units
recent_puma_sums_df = recent_units_df.groupby("PUMA").sum()
recent_puma_counts_df = recent_puma_sums_df[["rent_ELI_studio_count","rent_ELI_1_br_count",
    "rent_ELI_2_br_count",
    "rent_ELI_3_br_count",
    "rent_ELI_4_br_count",
    "rent_VLI_studio_count",
    "rent_VLI_1_br_count",
    "rent_VLI_2_br_count",
    "rent_VLI_3_br_count",
    "rent_VLI_4_br_count",
    "rent_LI_studio_count",
    "rent_LI_1_br_count",
    "rent_LI_2_br_count",
    "rent_LI_3_br_count",
    "rent_LI_4_br_count",
    "rent_MI_studio_count",
    "rent_MI_1_br_count",
    "rent_MI_2_br_count",
    "rent_MI_3_br_count",
    "rent_MI_4_br_count",
    "rent_MoI_studio_count",
    "rent_MoI_1_br_count",
    "rent_MoI_2_br_count",
    "rent_MoI_3_br_count",
    "rent_MoI_4_br_count",
    "rent_HI_studio_count",
    "rent_HI_1_br_count",
    "rent_HI_2_br_count",
    "rent_HI_3_br_count",
    "rent_HI_4_br_count",
    "own_ELI_studio_count",
    "own_ELI_1_br_count",
    "own_ELI_2_br_count",
    "own_ELI_3_br_count",
    "own_ELI_4_br_count",
    "own_VLI_studio_count",
    "own_VLI_1_br_count",
    "own_VLI_2_br_count",
    "own_VLI_3_br_count",
    "own_VLI_4_br_count",
    "own_LI_studio_count",
    "own_LI_1_br_count",
    "own_LI_2_br_count",
    "own_LI_3_br_count",
    "own_LI_4_br_count",
    "own_MI_studio_count",
    "own_MI_1_br_count",
    "own_MI_2_br_count",
    "own_MI_3_br_count",
    "own_MI_4_br_count",
    "own_MoI_studio_count",
    "own_MoI_1_br_count",
    "own_MoI_2_br_count",
    "own_MoI_3_br_count",
    "own_MoI_4_br_count",
    "own_HI_studio_count",
    "own_HI_1_br_count",
    "own_HI_2_br_count",
    "own_HI_3_br_count",
    "own_HI_4_br_count"]].copy()
recent_puma_counts_df

,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
PUMA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101,146.00,296.00,0.00,110.00,0.00,215.00,533.00,375.00,472.00,0.00,489.00,1470.00,1321.00,526.00,0.00,1296.00,2752.00,2032.00,782.00,81.00,1658.00,3425.00,2463.00,843.00,81.00,157.00,71.00,0.00,199.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,74.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
102,162.00,0.00,584.00,277.00,0.00,937.00,314.00,987.00,550.00,0.00,1625.00,1224.00,1939.00,1722.00,86.00,2408.00,2305.00,2848.00,1825.00,86.00,2652.00,3057.00,3251.00,1825.00,210.00,249.00,719.00,684.00,109.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
103,0.00,82.00,0.00,0.00,0.00,122.00,321.00,69.00,216.00,0.00,312.00,843.00,194.00,508.00,0.00,415.00,1524.00,428.00,776.00,112.00,415.00,1590.00,704.00,1110.00,140.00,0.00,567.00,60.00,63.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,282.00,0.00,0.00,0.00,55.00,282.00,0.00,0.00,0.00,0.00,0.00,0.00
104,0.00,622.00,81.00,237.00,0.00,0.00,658.00,587.00,401.00,0.00,248.00,1099.00,1053.00,995.00,0.00,248.00,1099.00,1248.00,1163.00,0.00,248.00,1207.00,1248.00,1163.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
105,0.00,99.00,0.00,0.00,0.00,0.00,416.00,281.00,66.00,206.00,541.00,1613.00,1040.00,251.00,378.00,685.00,2085.00,2092.00,460.00,378.00,795.00,2085.00,2442.00,520.00,513.00,64.00,68.00,330.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11103,0.00,208.00,65.00,0.00,0.00,103.00,208.00,65.00,342.00,0.00,359.00,712.00,1357.00,1005.00,674.00,520.00,1160.00,1405.00,1741.00,821.00,520.00,1160.00,1405.00,1820.00,821.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
11104,0.00,38.00,0.00,0.00,0.00,0.00,251.00,154.00,0.00,0.00,72.00,1066.00,2196.00,186.00,0.00,110.00,1806.00,2543.00,472.00,65.00,110.00,1806.00,2615.00,767.00,65.00,0.00,0.00,93.00,330.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
11105,40.00,0.00,0.00,0.00,121.00,76.00,250.00,256

In [27]:
#export it
recent_puma_counts_df.to_csv("weighted_recent_puma_counts.csv")